In [130]:
import numpy as np
import matplotlib.pyplot as plt

In [131]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.int)

In [132]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [133]:
X_st = X.copy().astype(np.float64)
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

### Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [134]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    err = - err / len(y)
    return err

In [135]:
y1 = np.array([1, 0, 0, 1, 0, 1])
y_pred1 = np.array([0.9, 0.1, 0.2, 0.8, 0.1, 0.9])
calc_logloss(y1, y_pred1)

0.14462152754328741

### Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [136]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [137]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#        if i % (iterations / 10) == 0:
#            print(i, W, err)
    return W, err

In [138]:
my_iterations = np.logspace(2, 4, 5, dtype=int)
my_coeff = np.linspace(1e-6, 10, 10)

In [139]:
my_err = 100
key_par = {}
for i in my_iterations:
    for e in my_coeff:        
        W, err = eval_model(X_st, y, iterations=i, alpha=e)
        if err < my_err:
            my_err = err
            key_par = {'iterations': i, 'alpha': e}
print(f'Лучшие параметры из предложенной выборки: \n{key_par} с логлоссом {my_err:.2f}')

Лучшие параметры из предложенной выборки: 
{'iterations': 10000, 'alpha': 10.0} с логлоссом 0.04


### Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [140]:
def calc_pred_proba(X, W):
    pred_proba = sigmoid(np.dot(W, X))
    return pred_proba

In [141]:
W, err = eval_model(X_st, y, iterations=100, alpha=1e-2)
W

array([ 0.30052426, -0.7012269 ,  0.66327117,  1.28844369])

In [142]:
y_pred_proba = calc_pred_proba(X_st, W)
y_pred_proba

array([0.55922256, 0.62517667, 0.76263659, 0.08090498, 0.80588097,
       0.79406299, 0.79864507, 0.19087155, 0.54230224, 0.81893585])

### Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [143]:
def calc_pred(X, W):
    y_pred_proba = calc_pred_proba(X_st, W)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [144]:
y_pred = calc_pred(X_st, W)
y_pred

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1])

### Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [145]:
print(y, y_pred, sep='\n')

[0 0 1 0 1 0 1 0 1 1]
[1 1 1 0 1 1 1 0 1 1]


In [155]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [156]:
print(f'accuracy = {accuracy(y, y_pred)}')

accuracy = 0.7


In [147]:
cm = np.array([['TP', 'FP'], ['FN', 'TN']])

In [149]:
def confusion_matrix(y, y_pred):
    confusion_matrix = np.zeros((2, 2))
    for i in range(len(y)):
        if y_pred[i] == 0:
            if y_pred[i] == y[i]:
                confusion_matrix[1][1] += 1
            else:
                confusion_matrix[1][0] += 1
        else:
            if y_pred[i] == y[i]:
                confusion_matrix[0][0] += 1
            else:
                confusion_matrix[0][1] += 1
    return confusion_matrix
    

In [150]:
print(cm, confusion_matrix(y, y_pred), sep='\n')

[['TP' 'FP']
 ['FN' 'TN']]
[[5. 3.]
 [0. 2.]]


In [159]:
def pre_rec_f1(y, y_pred):
    conf_m = confusion_matrix(y, y_pred)
    tp = conf_m[0][0]
    fp = conf_m[0][1]
    fn = conf_m[1][0]
    tn = conf_m[1][1]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [160]:
precision, recall, F1 = pre_rec_f1(y, y_pred)
print(f'precision = {precision},\nrecall = {recall},\nF1 = {F1:.2f}')

precision = 0.625,
recall = 1.0,
F1 = 0.77


### Могла ли модель переобучиться? Почему?

Могла, из-за того что набор данных очень маленький.